# **Predicting Disneyland Ride Popularity and Designing an Optimal Park Day Plan**

### **DSC 102 – Assignment 2**

**Author:**  
Kyle Le  

**Project Overview:**  
This project uses historic Disneyland wait-time data to predict ride popularity tiers (1–5) using machine learning models. The predicted popularity scores are then used to generate an optimized ride plan aimed at maximizing high-tier rides within a single park day.


## **1. Predictive Task**

### **Goal**

The goal of this project is to predict the **popularity tier** of each Disneyland ride using historic wait-time data.  
Each ride is assigned a popularity tier from **1 to 5**, where:

- **5** = most popular / in highest demand  
- **1** = least popular / lowest demand  

These tiers may be derived from my own personal tier list that I have created.

---

### **Problem Formulation**

This task is formulated as a **multiclass classification problem**:

- **Input features (X):**  
  Quantitative attributes of a ride, such as:  
  - Average wait time  
  - Maximum wait time  

- **Output label (y):**  
  A discrete popularity tier in **{1, 2, 3, 4, 5}**

The goal is to learn a function:

$$f(X) \rightarrow y$$

that accurately predicts the popularity tier from ride statistics.

---

### **Models to Be Used**

To align with course content, the project includes:

- A **majority-class baseline**  
- A **wait-time heuristic baseline** (ranking rides by average wait time)  
- A **multinomial logistic regression model**  
  - This is the primary ML model used for classification  
  - It outputs predicted probabilities for each tier  

---

### **Evaluation Strategy**

Model performance will be assessed using:

- **Accuracy**  
- **Macro F1-score**  
- **Confusion matrix visualization**  
  - Helps reveal whether errors occur mostly between adjacent tiers (e.g., 3 ↔ 4)

Performance of the logistic regression model will be compared against baselines to demonstrate prediction quality and justify the model’s effectiveness.
